1. 판례 내 사용 단어 빈도 분석

In [ ]:
import pandas as pd
from collections import Counter
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer

# 판례 데이터를 불러옴 (예시로 DataFrame 사용)
data = pd.read_csv("cases.csv")  # 판례 데이터를 불러온다고 가정
documents = data['content']  # 판례 내용이 담긴 열

# 한글 형태소 분석기 사용 (Okt)
okt = Okt()

# 모든 판례에서 명사만 추출
nouns = []
for doc in documents:
    nouns.extend(okt.nouns(doc))

# 빈도 분석
word_freq = Counter(nouns)
most_common_words = word_freq.most_common(20)  # 가장 많이 등장한 20개 단어

print(most_common_words)


2. 판례 내 단어 간 상관관계 분석

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import matplotlib.pyplot as plt

# TF-IDF Vectorizer로 문서-단어 행렬 생성
vectorizer = TfidfVectorizer(max_features=100)  # 상위 100개 단어로 제한
X = vectorizer.fit_transform(documents)

# 단어 간 상관관계 분석
correlation_matrix = (X.T * X).toarray()

# 상관관계 시각화
sns.heatmap(correlation_matrix, annot=False, cmap="coolwarm")
plt.show()


3. 판례 내 참조 법률 도출

In [ ]:
import re

# 참조 법률을 정규식으로 추출 (예시: "형법 제000조" 형태의 법 조항)
law_pattern = r'(형법\s*제\s*\d+조)'

laws = []
for doc in documents:
    found_laws = re.findall(law_pattern, doc)
    laws.extend(found_laws)

# 빈도 분석
law_freq = Counter(laws)
print(law_freq.most_common(20))


In [ ]:
4. 판례 내 참조 선례 도출

In [ ]:
# 판례에서 특정 선례를 참조하는 부분 찾기 (예시: "대법원 0000년 00월 00일 판결" 형태)
precedent_pattern = r'(대법원\s*\d+년\s*\d+월\s*\d+일\s*판결)'

precedents = []
for doc in documents:
    found_precedents = re.findall(precedent_pattern, doc)
    precedents.extend(found_precedents)

# 빈도 분석
precedent_freq = Counter(precedents)
print(precedent_freq.most_common(20))


5. 특정 법에 대한 판사별 사용 단어 분석

In [ ]:
# 판례 데이터에 판사별 정보가 포함되어 있다고 가정
# 특정 법에 대해 분석할 텍스트 필터링 (예시: '형법 제250조'가 포함된 판례만)
specific_law = '형법 제250조'
filtered_cases = data[data['content'].str.contains(specific_law)]

# 판사별로 텍스트 분석
judge_words = {}
for judge, group in filtered_cases.groupby('judge'):
    all_text = ' '.join(group['content'])
    nouns = okt.nouns(all_text)
    judge_words[judge] = Counter(nouns).most_common(10)  # 각 판사별 상위 10개 단어

# 결과 출력
for judge, words in judge_words.items():
    print(f"Judge: {judge}")
    print(f"Top words: {words}")
